In [1]:
import pandas as pd
import numpy as np
import random

class CPMM(object):
    def __init__(self, initial_state, fee):
        '''
        Parameters
        ----------
        initial_state   dict
                        dictionary containing the assets and initial liquidity of the market. Example: initial_state = {'A': 500, 'B': 500}

        fee             float
                        number between 0 and 1 representing the fixed fee of the CPMM
        '''
        self._state = initial_state.copy()
        self._fee = fee
        self._fee_inverse = 1.0 + fee
        self._fee_sum = []
        self._liquidity = {}
        self._book = []
        if (fee<0) | (fee>1):
            raise ValueError('Fee needs to be inside [0,1] interval')

    def get_prices(self, asset=None, compared_to=None):
        '''
        Parameters
        ----------
        asset           str
                        Optional. Name of the asset that you want to know the proce

        comparet_to     str
                        Optional. Name of the asset inside the pool that you want to compare with
        
        -------
        Returns         float/dict
                        Returns a float number in case that the name of the asset is provided.
                        If not, returns a dictionary with all the asset prices in the pool
        '''
        temp_dict = self._state.copy()
        del temp_dict['ZTG']
        
        if (asset is None) & (compared_to is None):
            prices = {}
            for key, value in temp_dict.items():
                prices[key] = (sum(temp_dict.values())-value)/(sum(temp_dict.values()))
        elif compared_to is None:
            prices = (sum(temp_dict.values())-temp_dict[asset])/sum(temp_dict.values())
        else:
            prices = temp_dict[compared_to]/temp_dict[asset]

        return prices


    def buy_shares(self, in_number, asset_out, asset_in='ZTG'):
        '''
        Parameters
        ----------
        in_number       int
                        Asset amount that is intended to put into the pool
        
        asset_out       float
                        Name of the asset that is taken out of the pool (bought)
        
        asset_in        float
                        Name of asset that is put into the pool (ZTG by default)

        -------
        Returns         float
                        Amount of assets that the pool is going to give away (fees discounted)
        '''
        k = self._state[asset_in] * self._state[asset_out]
        num_in = in_number / self._fee_inverse
        num_out = self._state[asset_out] - k / (self._state[asset_in] + num_in)
        self._state[asset_in] += in_number
        self._state[asset_out] -= num_out
        self._fee_sum += [in_number - num_in] #fee is expressed in ZTG
        self._book.append({'operation': 'buy',
                            'shares': num_out,
                            'outcome': asset_out,
                            'paid': in_number,
                            'fee': in_number - num_in
                        })
        return num_out

    def sell_shares(self, out_number, asset_in, asset_out='ZTG'):
        '''
        Parameters
        ----------
        out_number      int
                        Asset amount that is intended to extract outside the pool
        
        asset_in        float
                        Name of the asset that is taken out of the pool (sold)
        
        asset_out       float
                        Name of asset that is put into the pool (ZTG by default)

        -------
        Returns         float
                        Amount of assets that the pool is going to receive (fees discounted)
        '''
        k = self._state[asset_in] * self._state[asset_out]
        num_out = out_number/self._fee_inverse
        num_in = self._state[asset_in] - k / (self._state[asset_out] - num_out)
        self._state[asset_out] -= num_out
        self._state[asset_in] += num_in
        self._fee_sum += [out_number - num_out] 
        self._book.append({'operation': 'sell',
                            'shares': num_in,
                            'outcome': asset_in,
                            'paid': out_number,
                            'fee': out_number - num_out
                        })
        return num_in

    def provide_liquidity(self, id, amount):
        '''
        Use
        ---
        Determines the share distribution of a liquidity provider. They don't pay fee

        Parameters
        ----------
        id              int
                        Number that enables to identify each Liquidity Provider by separate
        
        amount          int
                        Total amount of ZTG provided

        -------
        Returns         dict
                        A dictionary with the quantity of shares provided plus the total amount of ZTG
        '''
        getprices = self.get_prices()
        liquidity_prividing = {}
        for key, value in getprices.items():
            liquidity_prividing[key] = value*amount
            self._state[key] += value*amount
        
        liquidity_prividing['total_provided'] = amount
        self._liquidity[id] = liquidity_prividing

        self._book.append({'operation': 'liquidity_providing',
                            'shares': amount,
                            'outcome': '-',
                            'paid': amount,
                            'fee': 0
                        })

        return liquidity_prividing

    @property
    def book(self):
        return pd.DataFrame(self._book)
    
    @property
    def state(self):
        return self._state

In [2]:
initial_state = {'A': 500, 'B': 500, 'ZTG':500}
cpmm = CPMM(initial_state = initial_state, fee = 0.03)

In [3]:
cpmm.buy_shares(10, asset_out='B', asset_in='ZTG')

9.523809523809518

In [4]:
cpmm.book

,operation,shares,outcome,paid,fee
0,buy,9.52381,B,10,0.291262


In [5]:
cpmm.state

{'A': 500, 'B': 490.4761904761905, 'ZTG': 510}

In [6]:
cpmm.get_prices()

{'A': 0.4951923076923077, 'B': 0.5048076923076923}

In [7]:
cpmm.sell_shares(out_number=10, asset_in='B', asset_out='ZTG')

-9.518264903477416

In [8]:
cpmm.book

,operation,shares,outcome,paid,fee
0,buy,9.523810,B,10,0.291262
1,sell,-9.518265,B,10,0.291262


In [9]:
cpmm.state

{'A': 500, 'B': 480.95792557271307, 'ZTG': 500.29126213592235}

In [10]:
cpmm.get_prices()

{'A': 0.49029414313760217, 'B': 0.5097058568623979}

In [11]:
cpmm.provide_liquidity(id=1, amount=1000)

{'A': 490.2941431376022, 'B': 509.70585686239787, 'total_provided': 1000}

In [12]:
cpmm.get_prices()

{'A': 0.5000932981192424, 'B': 0.4999067018807575}

In [13]:
cpmm.book

,operation,shares,outcome,paid,fee
0,buy,9.523810,B,10,0.291262
1,sell,-9.518265,B,10,0.291262
2,liquidity_providing,1000.000000,-,1000,0.000000


In [14]:
cpmm.get_prices()

{'A': 0.5000932981192424, 'B': 0.4999067018807575}

### Simulating 500 transactions

In [15]:
n = 500
outcome = 0.001
amount = np.random.random([n]) * 500.
outcomes = []
for i in range(n):
    outcomes += [random.choice(list(initial_state.keys()))]

outcomes = np.array(outcomes)

for i, a, o in zip(range(n),amount, outcomes):
    lista = [cpmm.buy_shares(a, asset_out= o), 
             cpmm.sell_shares(a, asset_in=o),
             cpmm.provide_liquidity(id= i, amount = a)]
    random.choice(lista)
    print(cpmm.get_prices())

{'A': 0.3186870601969089, 'B': 0.681312939803091}
{'A': 0.3451851150710335, 'B': 0.6548148849289666}
{'A': 0.39812124927235043, 'B': 0.6018787507276495}
{'A': 0.5606436480833993, 'B': 0.4393563519166007}
{'A': 0.553817390797728, 'B': 0.44618260920227215}
{'A': 0.5365332402010327, 'B': 0.46346675979896734}
{'A': 0.46740495387802855, 'B': 0.5325950461219714}
{'A': 0.38171647323374397, 'B': 0.618283526766256}
{'A': 0.2691135445313521, 'B': 0.7308864554686479}
{'A': 0.3635065705920611, 'B': 0.6364934294079388}
{'A': 0.4969542735076729, 'B': 0.503045726492327}
{'A': 0.49715376078742907, 'B': 0.5028462392125709}
{'A': 0.49723320225911805, 'B': 0.5027667977408818}
{'A': 0.5110869764432793, 'B': 0.48891302355672084}
{'A': 0.5378091500682444, 'B': 0.46219084993175574}
{'A': 0.6121564805076977, 'B': 0.38784351949230234}
{'A': 0.6808614345119834, 'B': 0.31913856548801656}
{'A': 0.6769826526107353, 'B': 0.32301734738926485}
{'A': 0.696742519415693, 'B': 0.30325748058430707}
{'A': 0.707677519575946

In [16]:
cpmm.book

,operation,shares,outcome,paid,fee
0,buy,9.523810,B,10.000000,0.291262
1,sell,-9.518265,B,10.000000,0.291262
2,liquidity_providing,1000.000000,-,1000.000000,0.000000
3,buy,393.479538,B,339.526717,9.889128
4,sell,-385.852492,B,339.526717,9.889128
...,...,...,...,...,...
1498,sell,611.974840,A,162.538285,4.734125
1499,liquidity_providing,162.538285,-,162.538285,0.000000
1500,buy,399.386645,ZTG,409.361584,11.923153
1501,sell,-395.508896,ZTG,409.361584,11.923153
